In [1]:
import numpy as np
import tensorflow as tf
import datetime
import torch
import torchvision
import torchvision.datasets as datasets
from torchvision import models
from torchvision import transforms

from matplotlib import pyplot as plt
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import LeakyReLU
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.datasets import mnist

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5), (0.5))])

batch_size = 500

trainset_MNIST_big = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainset_MNIST = torch.utils.data.Subset(trainset_MNIST_big,list(range(10000)))
trainloader_MNIST = torch.utils.data.DataLoader(trainset_MNIST, batch_size=batch_size, shuffle=True, num_workers=2)

val_set_MNIST = torch.utils.data.Subset(trainset_MNIST_big,list(range(10000, 20000)))
val_loader_MNIST = torch.utils.data.DataLoader(val_set_MNIST, batch_size=batch_size, shuffle=True, num_workers=2)
                                          
testset_MNIST = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader_MNIST = torch.utils.data.DataLoader(testset_MNIST, batch_size=batch_size, shuffle=False, num_workers=2)

In [3]:
# define model
model = models.resnet18(pretrained= False)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

In [4]:
def train(model, criterion, optimizer, train_loader, val_loader, num_epochs):
    best_model = model
    best_loss = 100
    for epoch in range(num_epochs):
        tr_correct = 0
        tr_total = 0
        val_correct = 0
        val_total = 0
        for batch_nr, (data, labels) in enumerate(train_loader):
            
            print("Epoch: ",epoch,"Batch: ",batch_nr)
            # calculate prediction according to our model
            
            prediction = model.forward(data)
            # Calculate the loss of the prediction by comparing to the expected output
            loss = criterion(prediction, labels)
            
            # Backpropagate the loss through the network to find the gradients of all parameters
            loss.backward()
            
            # Update the parameters along their gradients
            optimizer.step()
            
            # Clear stored gradient values
            optimizer.zero_grad()

            # calculate accuracy
            for i in range(len(data)):    
                guess = torch.argmax(prediction[i])
                if(guess.item() == labels[i]):
                    tr_correct+=1
                tr_total +=1

        for batch_nr, (data, labels) in enumerate(val_loader):
            
            prediction = model.forward(data)
            
            # Calculate the loss of the prediction by comparing to the expected output
            loss = criterion(prediction, labels)

            if(loss < best_loss):
                best_loss = loss
                best_model = model

            # calculate accuracy
            for i in range(len(data)):    
                guess = torch.argmax(prediction[i])
                if(guess.item() == labels[i]):
                    val_correct+=1
                val_total +=1

    # primt accuracy
    tr_accuracy = tr_correct/tr_total
    val_accuracy = val_correct/val_total
    print(f'Training accuracy:   {str(100*tr_accuracy)[:4]}%.')
    print(f'Validation accuracy: {str(100*val_accuracy)[:4]}%.')

    return best_model

In [5]:
def test(model, test_loader):
    val_correct = 0
    val_total = 0
    for batch_nr, (data, labels) in enumerate(test_loader):
        prediction = model.forward(data)
        
        # calculate accuracy
        for i in range(len(data)):    
            guess = torch.argmax(prediction[i])
            if(guess.item() == labels[i]):
                val_correct+=1
            val_total +=1

    # primt accuracy
    val_accuracy = val_correct/val_total
    print(f'Test accuracy: {str(100*val_accuracy)[:4]}%.')

In [6]:
Trained_model = train(model, criterion, optimizer, trainloader_MNIST, val_loader_MNIST, num_epochs=10)
test(Trained_model, testloader_MNIST)

Epoch:  0 Batch:  0
Epoch:  0 Batch:  1
Epoch:  0 Batch:  2
Epoch:  0 Batch:  3
Epoch:  0 Batch:  4
Epoch:  0 Batch:  5
Epoch:  0 Batch:  6
Epoch:  0 Batch:  7
Epoch:  0 Batch:  8
Epoch:  0 Batch:  9
Epoch:  0 Batch:  10
Epoch:  0 Batch:  11
Epoch:  0 Batch:  12
Epoch:  0 Batch:  13
Epoch:  0 Batch:  14
Epoch:  0 Batch:  15
Epoch:  0 Batch:  16
Epoch:  0 Batch:  17
Epoch:  0 Batch:  18
Epoch:  0 Batch:  19
Epoch:  1 Batch:  0
Epoch:  1 Batch:  1
Epoch:  1 Batch:  2
Epoch:  1 Batch:  3
Epoch:  1 Batch:  4
Epoch:  1 Batch:  5
Epoch:  1 Batch:  6
Epoch:  1 Batch:  7
Epoch:  1 Batch:  8
Epoch:  1 Batch:  9
Epoch:  1 Batch:  10
Epoch:  1 Batch:  11
Epoch:  1 Batch:  12
Epoch:  1 Batch:  13
Epoch:  1 Batch:  14
Epoch:  1 Batch:  15
Epoch:  1 Batch:  16
Epoch:  1 Batch:  17
Epoch:  1 Batch:  18
Epoch:  1 Batch:  19
Epoch:  2 Batch:  0
Epoch:  2 Batch:  1
Epoch:  2 Batch:  2
Epoch:  2 Batch:  3
Epoch:  2 Batch:  4
Epoch:  2 Batch:  5
Epoch:  2 Batch:  6
Epoch:  2 Batch:  7
Epoch:  2 Batch:  8


In [7]:
torch.save(Trained_model, './CNN_model')

In [8]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 500

all_svhn_train_data = torchvision.datasets.SVHN(root='./data', split= 'train', download=True, transform=transform)
all_svhn_test_data = torchvision.datasets.SVHN(root='./data', split= 'test', download=True, transform=transform)

svhn_train_set = torch.utils.data.Subset(all_svhn_train_data,list(range(10000)))
svhn_val_set  = torch.utils.data.Subset(all_svhn_train_data,list(range(10000, 20000)))
svhn_test_set  = torch.utils.data.Subset(all_svhn_test_data,list(range(10000)))

svhn_train_loader = torch.utils.data.DataLoader(svhn_train_set, batch_size=batch_size, shuffle=False, num_workers=2)
svhn_val_loader = torch.utils.data.DataLoader(svhn_val_set, batch_size=batch_size, shuffle=False, num_workers=2)
svhn_test_loader = torch.utils.data.DataLoader(svhn_test_set, batch_size=batch_size, shuffle=False, num_workers=2)

182041600it [00:25, 7267607.79it/s]                               


64275456it [00:10, 6095374.53it/s]                               


In [9]:
model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
test(Trained_model, svhn_test_loader)

Test accuracy: 10.3%.


In [10]:

#mnistCNN = torch.load('./CNN_model')

trained_resnet_fe = Trained_model 

# FREEZE all old params
for param in trained_resnet_fe.parameters():
    param.requires_grad = False

# new layer
num_ftrs = trained_resnet_fe.fc.in_features
trained_resnet_fe.fc = nn.Linear(num_ftrs, 10)

# find the paramaters we want to update during training
params_to_update = []
for param in trained_resnet_fe.parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

# Define our loss function
criterion_fe = torch.nn.CrossEntropyLoss()

# Define our optimizer
optimizer_fe = torch.optim.Adam(params_to_update, lr=0.001)

In [11]:
# Run the training step
train(trained_resnet_fe, criterion_fe, optimizer_fe, svhn_train_loader, svhn_val_loader, num_epochs=10)

Epoch:  0 Batch:  0
Epoch:  0 Batch:  1
Epoch:  0 Batch:  2
Epoch:  0 Batch:  3
Epoch:  0 Batch:  4
Epoch:  0 Batch:  5
Epoch:  0 Batch:  6
Epoch:  0 Batch:  7
Epoch:  0 Batch:  8
Epoch:  0 Batch:  9
Epoch:  0 Batch:  10
Epoch:  0 Batch:  11
Epoch:  0 Batch:  12
Epoch:  0 Batch:  13
Epoch:  0 Batch:  14
Epoch:  0 Batch:  15
Epoch:  0 Batch:  16
Epoch:  0 Batch:  17
Epoch:  0 Batch:  18
Epoch:  0 Batch:  19
Epoch:  1 Batch:  0
Epoch:  1 Batch:  1
Epoch:  1 Batch:  2
Epoch:  1 Batch:  3
Epoch:  1 Batch:  4
Epoch:  1 Batch:  5
Epoch:  1 Batch:  6
Epoch:  1 Batch:  7
Epoch:  1 Batch:  8
Epoch:  1 Batch:  9
Epoch:  1 Batch:  10
Epoch:  1 Batch:  11
Epoch:  1 Batch:  12
Epoch:  1 Batch:  13
Epoch:  1 Batch:  14
Epoch:  1 Batch:  15
Epoch:  1 Batch:  16
Epoch:  1 Batch:  17
Epoch:  1 Batch:  18
Epoch:  1 Batch:  19
Epoch:  2 Batch:  0
Epoch:  2 Batch:  1
Epoch:  2 Batch:  2
Epoch:  2 Batch:  3
Epoch:  2 Batch:  4
Epoch:  2 Batch:  5
Epoch:  2 Batch:  6
Epoch:  2 Batch:  7
Epoch:  2 Batch:  8


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [12]:
print("Feature Extraction of MNIST-trained Resnet on SVHN dataset:")
test(trained_resnet_fe, svhn_test_loader)

Feature Extraction of MNIST-trained Resnet on SVHN dataset:
Test accuracy: 26.0%.
